In [1]:
import pandas as pd
from pathlib import Path

In [27]:
interim = '../data/interim'
people = pd.read_pickle(Path(interim) / 'people.pkl')
directory = pd.read_pickle(Path(interim) / 'directory.pkl')

In [29]:
people.dtypes

playerID         object
birthYear       float64
birthMonth      float64
birthDay        float64
birthCountry     object
birthState       object
birthCity        object
deathYear       float64
deathMonth      float64
deathDay        float64
deathCountry     object
deathState       object
deathCity        object
nameFirst        object
nameLast         object
nameGiven        object
weight          float64
height          float64
bats             object
throws           object
debut            object
finalGame        object
retroID          object
bbrefID          object
dtype: object

In [25]:
directory.groupby(['FirstName', 'LastName']).size()

FirstName  LastName 
AJ         Achter       1
           Burnett      1
           Cole         1
           Ellis        1
           Griffin      1
                       ..
Zinn       Beck         1
Zip        Collins      1
           Zabel        1
Zoilo      Almonte      1
           Versalles    1
Length: 19232, dtype: int64

In [20]:
directory.head()

,FirstName,LastName,BattingHand
PLAYER_ID,,,
aardd001,David,Aardsma,R
aaroh101,Hank,Aaron,R
aarot101,Tommie,Aaron,R
aased001,Don,Aase,R
abada001,Andy,Abad,L


In [15]:
people = people[['playerID', 'nameFirst', 'nameLast']]
people.columns = ['NewID', 'FirstName', 'LastName']

In [19]:
pd.merge(people, directory, on=['FirstName', 'LastName'], how='outer', indicator = True)

,NewID,FirstName,LastName,BattingHand,_merge
0,aardsda01,David,Aardsma,R,both
1,aaronha01,Hank,Aaron,R,both
2,aaronto01,Tommie,Aaron,R,both
3,aasedo01,Don,Aase,R,both
4,abadan01,Andy,Abad,L,both
...,...,...,...,...,...
21866,NaN,Joe,Woerlin,?,right_only
21867,NaN,Bill,Wright,?,right_only
21868,NaN,Frank,Wyman,?,right_only
21869,NaN,JB,Young,?,right_only


In [ ]:

rosters = pd.read_pickle(clean_rosters)
roster2019 = rosters[rosters.Year == self.__year]

raw_lineup_wide = self.__raw_data

raw_lineup_wide.drop(
    columns=['home_team_name', 'away_team_name'], inplace=True)

raw_lineup_wide['id'] = np.arange(len(raw_lineup_wide))

raw_lineup = raw_lineup_wide.melt(
    id_vars=['id', 'home_team_code', 'away_team_code'],
    var_name='lineup_id',
    value_name='Name')

# Get First and Last Names
raw_lineup['FirstName'] = raw_lineup.Name.str.split().str.get(0)
raw_lineup['FirstName'] = \
    raw_lineup['FirstName'].str.replace('.', '', regex=False)
raw_lineup['LastName'] = raw_lineup.Name.str.split().str.get(1)
raw_lineup['LastName'] = \
    raw_lineup['LastName'].str.replace('.', '', regex=False)
raw_lineup.drop(columns=['Name'], inplace=True)

# set whether players are at home or away, and spot in lineup
raw_lineup[['home', 'lineup_id']] = \
    raw_lineup['lineup_id'].str.split(pat='_', expand=True)

# set team variable for each player
raw_lineup['TEAM'] = raw_lineup['away_team_code']
raw_lineup.loc[raw_lineup.home == 'home', 'TEAM'] = \
    raw_lineup['home_team_code']

# set up record linking
thresh = 0.4
indexer = recordlinkage.Index()
indexer.block(['TEAM', 'LastName'])
pairs = indexer.index(raw_lineup, roster2019)

# Generate matches
compare = recordlinkage.Compare()
compare.exact('TEAM', 'TEAM', label='Team')
compare.exact('LastName', 'LastName', label='LastName')
compare.string('FirstName', 'FirstName',
               threshold=thresh, label='FirstName')
features = compare.compute(pairs, raw_lineup, roster2019)

# keep best matches
lineup_with_id = features[features.sum(axis=1) >= 3]
lineup_with_id = lineup_with_id.reset_index()[['level_0', 'level_1']]

# merge matches
lineup_with_id = lineup_with_id.join(
    roster2019, on='level_1', rsuffix='_roster')
lineup_with_id = lineup_with_id.join(
    raw_lineup, on='level_0', rsuffix='_lineup')

# clean up matched data
lineup_with_id = lineup_with_id[[
    'PLAYER_ID', 'TEAM', 'lineup_id', 'home', 'id',
    'home_team_code', 'away_team_code'
]]
lineup_with_id = lineup_with_id.set_index([
    'id', 'home_team_code', 'away_team_code', 'home', 'lineup_id'])
lineup_with_id = lineup_with_id.reset_index()

players_grid = lineup_with_id.pivot(
    index=['id', 'home_team_code', 'away_team_code'],
    columns=['lineup_id', 'home'],
    values=['PLAYER_ID'])
players_grid.columns = players_grid.columns.droplevel()
players_grid.columns = players_grid.columns.swaplevel(0, 1)
players_grid.columns = [
    '_'.join(col).strip() for col in players_grid.columns.values
]
players_grid = players_grid.set_index(
    ['home_pitcher', 'away_pitcher'], append=True
)

clean_lineups = players_grid.stack().reset_index().set_index([
    'home_team_code', 'away_team_code'
])
clean_lineups.drop(columns=['id'], inplace=True)
clean_lineups.columns = [
    'home_pitcher', 'away_pitcher', 'spot', 'BAT_ID'
]
clean_lineups[['home', 'spot']] = \
    clean_lineups['spot'].str.split(pat='_', expand=True)
clean_lineups['spot'] = clean_lineups['spot'].str.slice(start=-1)

d = {'home': True, 'away': False}
clean_lineups['home'] = clean_lineups['home'].map(d)
clean_lineups

self.__clean_data = clean_lineups